In [24]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.stem import WordNetLemmatizer
import urllib2
import pandas as pd

%matplotlib inline

In [25]:
SF = pd.read_csv('SF_jobs_new.csv')

In [29]:

def text_cleaner(html,Oneline=True): # return a single line of cleaned text

    soup_obj = BeautifulSoup(html,"lxml") # Get the html from the site

    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    
    text = soup_obj.body.get_text('\n') # Get the text from this

    lines = (line.strip() for line in text.splitlines()) # break into lines
#    lines = [line for i, line in enumerate(lines) if i>16]
#    for line in lines:
#        print line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    def chunk_space(chunk):
        chunk_out = chunk + ' ' # Need to fix spacing issue
        return chunk_out  

    text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line

    # Now clean out all of the unicode junk (this line works great!!!)
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception

    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                                # Also include + for C++
        
    # remove the junk from the beginning and end parts of Indeed
    try:
        text = ((text.lower()).split('advanced search')[1]).split('save resume')[0]
    except:
        return text
    
    return text

In [28]:
SF.job_description[0]

'<div class="highlight-black" id="jobdescSec">\n<p><strong>The Opportunity:</strong></p><p>Are you passionate about mining data\xc2\xa0for answers\xc2\xa0to challenging problem, finding actionable solutions\xc2\xa0that\xc2\xa0no one else can solve.\xc2\xa0Do you thrive at distilling\xc2\xa0large quantities of customer\xc2\xa0data\xc2\xa0into a coherent story with clear, easy to follow conclusions,\xc2\xa0implications\xc2\xa0and actionable recommendations informed by knowledge of the business? If this\xc2\xa0sounds like you,\xc2\xa0please continue\xc2\xa0reading.\xc2\xa0\xc2\xa0</p><p>The\xc2\xa0Rodan\xc2\xa0+ Fields Business Intelligence and Analytics Department is looking for a Data Scientist with a specialty in customer analytics to join the Advanced Analytics team. Members of our team work closely with Sales, Marketing, and Customer Loyalty\xc2\xa0providing insights\xc2\xa0into the behavior of R&amp;F\xe2\x80\x99s customers\xc2\xa0to enable the leadership and operational teams make 

In [31]:
text_cleaner(SF.job_description[0],Oneline=True)

'The Opportunity  Are you passionate about mining datafor answersto challenging problem  finding actionable solutionsthatno one else can solve Do you thrive at distillinglarge quantities of customerdatainto a coherent story with clear  easy to follow conclusions implicationsand actionable recommendations informed by knowledge of the business  If thissounds like you please continuereading  TheRodan+ Fields Business Intelligence and Analytics Department is looking for a Data Scientist with a specialty in customer analytics to join the Advanced Analytics team  Members of our team work closely with Sales  Marketing  and Customer Loyaltyproviding insightsinto the behavior of R Fs customersto enable the leadership and operational teams make better informed  data driven decisions The ideal candidate will bring analytical rigor and statistical methods to the challenges of understandingand modelingthe behavior of R Fs consultants and customers  The insights from this role willbedrivers of multi

In [86]:
def text_preprocess(line): # perform tokenization, select noun, Lemmertization etc on a line text

    rtext=[]    
    for w, tag in nltk.pos_tag(nltk.word_tokenize(line.lower())):  # Tokenization
        if tag in ['NN']:  # Keep only Nouns (for this special case)
            rtext.append(w)
    
    wordnet_lemmatizer = WordNetLemmatizer()  # Lemmertization
    rtext = [wordnet_lemmatizer.lemmatize(w) for w in rtext]
            
    stop_words = set(stopwords.words("english")) # Filter out any stop words
    rtext = [w for w in rtext if not w in stop_words]
    
    return rtext

In [87]:
texts = []
for i in range (SF.shape[0]):
    texts.append(text_preprocess(SF.job_description[i]))

In [54]:
dictionary = corpora.Dictionary(texts)

In [57]:
corpus = [dictionary.doc2bow(text) for text in texts]


In [59]:
tfidf = models.TfidfModel(corpus)

In [60]:
corpus_tfidf = tfidf[corpus]

In [14]:
def TFIDFModels(corpus_list):
  tfidf = models.TfidfModel(corpus_list)
  tfidf_corpus = tfidf[corpus_list]
  return tfidf,tfidf_corpus

In [15]:
#Create a bag of words from a list of text 
def GetVectors(evt_list,max_features=500):
  vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = max_features) 
  in_features = vectorizer.fit_transform(evt_list)
  in_features = in_features.toarray()
  vocab = vectorizer.get_feature_names()
  vocab = numpy.array(vocab)
  return in_features,vocab

In [82]:
TFIDFModels(corpus)

(<gensim.models.tfidfmodel.TfidfModel at 0x1221ab050>,
 <gensim.interfaces.TransformedCorpus at 0x121546c50>)